<a href="https://colab.research.google.com/github/andresalfp/Wedge-limit-equilibrium-factor-of-security/blob/main/wedge_fs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
# Importing libraries
from __future__ import annotations
import numpy as np

# Utils
def norm_ddir2cart(dip: int | float, dipdir: int | float) -> tuple:
  """
  Calculates cartesian coordinates of plane-normal vector, given plane in dip-dip direction notation.

  Parameters:
  -----------
  dip: int or float
    Dip of plane.

  dipdir: int or float
    Dip direction of plane.

  Returns:
  -----------
  tuple with x, y and z coordinates of plane-normal vector.
  """
  dip = np.deg2rad(dip)
  dipdir = np.deg2rad(dipdir)

  x = np.sin(dip)*np.sin(dipdir)
  y = np.sin(dip)*np.cos(dipdir)
  z = np.cos(dip)

  return x, y, z


def ddir2cart(dip: int | float, dipdir: int | float) -> tuple:
  """
  Calculates cartesian coordinates of plane-maximum dip vector, given plane in dip-dip direction notation.

  Parameters:
  -----------
  dip: int or float
    Dip of plane.

  dipdir: int or float
    Dip direction of plane.

  Returns:
  -----------
  tuple with x, y and z coordinates of plane-maximum dip vector.
  """
  dip = np.deg2rad(dip + 90)
  dipdir = np.deg2rad(dipdir)

  x = np.sin(dip)*np.sin(dipdir)
  y = np.sin(dip)*np.cos(dipdir)
  z = np.cos(dip)

  return x, y, z


def cart2ddir(x: int | float, y: int | float, z: int | float) -> tuple:
  """
  Calculates dip-dip direction notation values of vector given in cartesian coordinates.

  Parameters:
  -----------
  x: int or float
    x-coordinate of vector.

  y: int or float
    y-coordinate of vector.

  z: int or float
    z-coordinate of vector.

  Returns:
  -----------
  tuple with dip and dip direction values of vector.

  """
  r = np.sqrt(x**2 + y**2 + z**2)

  x = x/r
  y = y/r
  z = z/r

  dip = np.arccos(z)
  dipdir = np.arctan(np.divide(x, y))

  if dip >=0 and dipdir >= 0:
    if z >= 0:
      return 90 - np.rad2deg(dip), np.rad2deg(dipdir) + 180

    else:
      return 90 - np.rad2deg(dip), np.rad2deg(dipdir)
  
  if dip >=0 and dipdir < 0:
    return 90 - np.rad2deg(dip), np.rad2deg(dipdir) + 180

  if dip < 0 and dipdir >=0:
    return 90 - np.rad2deg(dip), np.rad2deg(dipdir) + 360


def planes_angle(dips: list | tuple, dipdirs: list | tuple) -> np.array:
  """
  Calculates angle between 2 planes, given planes in dip-dip direction notation.

  Parameters:
  -----------
  dips: list or tuple
    Dip values of planes.

  dipdirs: list or tuple
    Dip direction values of planes.

  Returns:
  -----------
  np.array with dip and dip direction values of vector.
  """
  p1 = norm_ddir2cart(dips[0], dipdirs[0])
  p2 = norm_ddir2cart(dips[1], dipdirs[1])

  return np.arccos(np.dot(a=p1, b=p2))


def lines_angle(plunges: list | tuple, strikes: list | tuple) -> int | float:
  """
  Calculates angle between 2 vectors, given vectors in plunge-strike notation.

  Parameters:
  -----------
  plunges: list or tuple
    Plunge values of lines.

  strikes: list or tuple
    Strike values of lines.

  Returns:
  -----------
  int or float values of angle between 2 vectors.
  """
  line_1 = ddir2cart(plunges[0], strikes[0])
  line_2 = ddir2cart(plunges[1], strikes[1])

  proj = np.dot(line_1, line_2)

  theta = np.arccos(np.divide(proj,(np.linalg.norm(line_1)*np.linalg.norm(line_2))))

  return theta


def planes_intersection(dips: list | tuple, dipdirs: list | tuple) -> np.array:
  """
  Calculates cartesian coordinates of intersection line formed by 2 planes, given planes in dip-dip direction notation.

  Parameters:
  -----------
  dips: list or tuple
    Dip values of planes.

  dipdirs: list or tuple
    Dip direction values of planes.

  Returns:
  -----------
  np.array with x, y and z values of intersection line formed by 2 planes.

  """
  p1 = norm_ddir2cart(dips[0], dipdirs[0])
  p2 = norm_ddir2cart(dips[1], dipdirs[1])

  normal = np.cross(a=p1, b=p2)

  return normal


# Wedge factor of security function

def wedge_fs(dips: list | tuple, dipdirs: list | tuple, friction: int | float) -> int | float:
  """
  Calculates factor of security of wedge formed by the intersection of two planes (assuming the existence of 1 additional free-face).

  Parameters:
  -----------
  dips: list or tuple
    Dip values of planes.

  dipdirs: list or tuple
    Dip direction values of planes.

  friction: int or float
    Friction angle of discontinuity formed by the dicontinuities in rock mass.

  Returns:
  -----------
  int or float value of limit-equilibrium factor of security of wedge formed by the intersection of two planes.
  """
  friction = np.deg2rad(friction)
  
  intersection_line = planes_intersection(dips, dipdirs)
  cart_intersection_line = cart2ddir(intersection_line[0], intersection_line[1], intersection_line[2])

  il_plunge = cart_intersection_line[0]
  
  epsilon = planes_angle(dips, dipdirs)
  
  aux_plane_azimuth = 0, cart_intersection_line[1] + 90
  beta = lines_angle([intersection_line[0], aux_plane_azimuth[0]], [intersection_line[1], aux_plane_azimuth[1]])

  num = np.sin(beta)*np.tan(friction)
  den = np.sin(epsilon/2)*np.tan(il_plunge)

  return np.abs(np.divide(num, den))


In [72]:
# Dip-dip direction of Plane 1
dip_1 = 45
dipdir_1 = 120 

# Dip-dip direction of Plane 2
dip_2 = 80
dipdir_2 = 75

# Friction angle
friction = 20

# Wedge factor of security
fs = wedge_fs(dips=[dip_1, dip_2], dipdirs=[dipdir_1, dipdir_2], friction=friction)
print('Wedge limit-equilibrium factor of security: {:.3f}'.format(fs))

Wedge limit-equilibrium factor of security: 0.541
